In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import re
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torchtext
from tqdm import tqdm
from transformers import pipeline, BertConfig, BertForSequenceClassification, BertTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Data Loading

In [ ]:
directory = '/content/drive/MyDrive/Meme Data/bully_data'
import pandas as pd

df = pd.read_csv("/content/Copy of Cyberbully_corrected_emotion_sentiment - cyberbully.csv")
df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target,Unnamed: 10,Unnamed: 11
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual,NaN,NaN
1,1.jpg,Aaloo ke paranthe is the best breakfast Omelet...,Nonbully,Nonbully,Nonbully,Neutral,Other,No,Harmless,NaN,NaN,NaN
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society,NaN,NaN
3,3.jpg,You find a new YouTuber He's funny All of his ...,Nonbully,Nonbully,Nonbully,Neutral,Sadness,No,Harmless,NaN,NaN,NaN
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual,NaN,NaN


## Data Preprocessing

In [ ]:
df.dropna(subset=['Img_Text', 'Img_Text_Label'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Img_Text_Label'].value_counts()

Img_Text_Label
Bully       3222
Nonbully    2632
Name: count, dtype: int64

In [ ]:
import os
valid_imgs = os.listdir(directory)
data = []
labels = {"Nonbully":0, "Bully":1}
for index, row in df.iterrows():
    img_id = row["Img_Name"]
    caption = row["Img_Text"]
    label = row["Img_Text_Label"]
    if img_id in valid_imgs:
        data.append({"ImageID":img_id, "Labels":labels[label], "Caption":caption})

In [ ]:
new_df = pd.DataFrame(data)
new_df.head()

,ImageID,Labels,Caption
0,0.jpg,1,Shivam @shivamishraa Girls be named naina and ...
1,1.jpg,0,Aaloo ke paranthe is the best breakfast Omelet...
2,2.jpg,1,For Boyfriend For Bestfriend DESI ADUKT TROLLS
3,3.jpg,0,You find a new YouTuber He's funny All of his ...
4,4.jpg,1,not_shubham14 @mentally_dank Kids at Marine Dr...


In [ ]:
new_df['Labels'].value_counts()

Labels
1    3193
0    2605
Name: count, dtype: int64

In [ ]:
new_df.shape

(5798, 3)

In [ ]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download("stopwords")
import re

punctuation = list(string.punctuation)
stopwords = list(stopwords.words('english'))
stop = stopwords + punctuation

def lowerizer(article):
    return article.lower()

def remove_html(article):
    article = re.sub("()", "", article, flags=re.DOTALL)
    return article

def remove_url(article):
    article = re.sub(r'https?:\/\/.\S+', "", article)
    return article

def remove_hashtags(article):
    article = re.sub("#"," ",article)
    return article

def remove_a(article):
    article = re.sub("@"," ",article)
    return article

def remove_brackets(article):
    article = re.sub('', '', article)
    return article

def remove_stop_punct(article):
    final_article = []
    for i in article.split():
        if i not in stop:
            final_article.append(i.strip())
    return " ".join(final_article)

def preprocessing(article):
    article = lowerizer(article)
    article = remove_html(article)
    article = remove_url(article)
    article = remove_hashtags(article)
    article = remove_a(article)
    article = remove_brackets(article)
    article = remove_stop_punct(article)
    return article

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
new_df["Caption"] = new_df["Caption"].apply(lambda x: preprocessing(x))
new_df.head()

,ImageID,Labels,Caption
0,0.jpg,1,shivam shivamishraa girls named naina eyes work
1,1.jpg,0,aaloo ke paranthe best breakfast omelette best...
2,2.jpg,1,boyfriend bestfriend desi adukt trolls
3,3.jpg,0,find new youtuber he's funny videos funny last...
4,4.jpg,1,not_shubham14 mentally_dank kids marine drive ...


## Caption Extraction

In [ ]:
pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
def img2text(img_id):
  # Salesforce/blip2-opt-2.7b   Salesforce/blip-image-captioning-large
  url = f"{directory}/{img_id}"
  text = pipe(url)[0]["generated_text"]
  return text

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

In [ ]:
img2text(new_df["ImageID"][0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a group of women with glasses on their faces'

In [ ]:
# img_captions = []
len(valid_imgs)

5949

In [ ]:
new_df.shape

(5798, 3)

In [ ]:
caption_df = pd.read_csv("/content/drive/MyDrive/Meme Data/cap.csv")
caption_df

,0
0,a group of women with glasses on their faces
1,a bunch of dragons
2,a woman taking a self self self self self self...
3,a man in a suit and tie is speaking into a mic...
4,a screenshot of a man in yellow shirt and sung...
...,...
5793,a person getting injection from a person
5794,a woman with a quote on her face
5795,a woman in a bra bra bra bra bra bra bra bra b...
5796,a bounce castle with a stuffed bear on top


In [ ]:
img_captions = caption_df['0'].values.tolist()

In [ ]:
len(img_captions)

5798

In [ ]:
# from tqdm import tqdm

# for i in tqdm(range(5000, 5798)):
#   cap = img2text(new_df["ImageID"][i])
#   img_captions.append(cap)
#   cap_df = pd.DataFrame(img_captions)
#   cap_df.to_csv("/content/drive/MyDrive/Meme Data/cap.csv", index=False)

  0%|          | 0/798 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 798/798 [1:32:05<00:00,  6.92s/it]


In [ ]:
data = pd.concat([new_df, caption_df], axis=1)
data

,ImageID,Labels,Caption,0
0,0.jpg,1,shivam shivamishraa girls named naina eyes work,a group of women with glasses on their faces
1,1.jpg,0,aaloo ke paranthe best breakfast omelette best...,a bunch of dragons
2,2.jpg,1,boyfriend bestfriend desi adukt trolls,a woman taking a self self self self self self...
3,3.jpg,0,find new youtuber he's funny videos funny last...,a man in a suit and tie is speaking into a mic...
4,4.jpg,1,not_shubham14 mentally_dank kids marine drive ...,a screenshot of a man in yellow shirt and sung...
...,...,...,...,...
5793,6001.jpg,1,corona vaccine ki testing bezubaan janwar par ...,a person getting injection from a person
5794,6002.jpg,0,"reply everyone instantly?"" know much hurts som...",a woman with a quote on her face
5795,6003.jpg,1,girls reading desi adult trolls memes,a woman in a bra bra bra bra bra bra bra bra b...
5796,6004.jpg,0,jumping would solve problems,a bounce castle with a stuffed bear on top


In [ ]:
data.isnull().sum()

ImageID    0
Labels     0
Caption    0
0          0
dtype: int64

## Special Data

In [ ]:
test_df = df.loc[(df['Img_Label'] == 'Nonbully') & (df['Text_Label'] == 'Nonbully') & (df['Img_Text_Label'] == 'Bully')]
test_df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target,Unnamed: 10,Unnamed: 11
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual,NaN,NaN
10,10.png,when your debit card decline at the abortion c...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Harmless,NaN,NaN,NaN
12,12.png,how you guys think you looks like but when pad...,Bully,Nonbully,Nonbully,Negative,Disgust,No,Partially-Harmful,Individual,NaN,NaN
14,14.png,when you forget to wash your hands,Bully,Nonbully,Nonbully,Positive,Anticipation,No,Harmless,NaN,NaN,NaN
17,17.jpg,i want you to make my legs shake,Bully,Nonbully,Nonbully,Negative,Disgust,Yes,Partially-Harmful,Individual,NaN,NaN


In [ ]:
test_df

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target,Unnamed: 10,Unnamed: 11
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual,NaN,NaN
10,10.png,when your debit card decline at the abortion c...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Harmless,NaN,NaN,NaN
12,12.png,how you guys think you looks like but when pad...,Bully,Nonbully,Nonbully,Negative,Disgust,No,Partially-Harmful,Individual,NaN,NaN
14,14.png,when you forget to wash your hands,Bully,Nonbully,Nonbully,Positive,Anticipation,No,Harmless,NaN,NaN,NaN
17,17.jpg,i want you to make my legs shake,Bully,Nonbully,Nonbully,Negative,Disgust,Yes,Partially-Harmful,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5832,5984.jpg,March 2020 March 2021,Bully,Nonbully,Nonbully,Negative,Sadness,Yes,Partially-Harmful,NaN,NaN,NaN
5837,5989.jpg,32 DEC 2019 South Sudan has refused to enter 2...,Bully,Nonbully,Nonbully,Neutral,Surprise,Yes,Harmless,Community,NaN,NaN
5840,5992.jpg,"If You Have This , You're My Crush Xavier I al...",Bully,Nonbully,Nonbully,Negative,Disgust,Yes,Harmless,NaN,NaN,NaN
5843,5995.jpg,Applying for jobs in the middle of a pandemic,Bully,Nonbully,Nonbully,Neutral,Surprise,Yes,Harmless,NaN,NaN,NaN


## Dataset Creation

In [ ]:
train_data, valid_data = train_test_split(data, test_size=0.1, random_state=42)
train_data = train_data.reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)

In [ ]:
train_data["Labels"].value_counts()

Labels
1    2891
0    2327
Name: count, dtype: int64

In [ ]:
valid_data["Labels"].value_counts()

Labels
1    302
0    278
Name: count, dtype: int64

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.model_max_length

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

512

In [ ]:
class Meme_Dataset(torch.utils.data.Dataset):
  def __init__(self, dataframe, tokenizer):
    self.df = dataframe
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    img_id = self.df.loc[index, "ImageID"]
    img_caption = self.df.loc[index, '0']

    caption = self.df.loc[index, "Caption"]
    text = img_caption +" " + caption

    encoded_caption = self.tokenizer.encode_plus(
            text,
            add_special_tokens = True,
            max_length = 256,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors = 'pt',
            padding = "max_length",
            truncation = True
        )

    input_ids = encoded_caption['input_ids'][0]
    attention_mask = encoded_caption['attention_mask'][0]

    label = self.df.loc[index, "Labels"]
    label_tensor = torch.tensor(label, dtype=torch.float)
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': label_tensor
    }


In [ ]:
train_dataset = Meme_Dataset(train_data, tokenizer)
val_dataset = Meme_Dataset(valid_data, tokenizer)

In [ ]:
BATCH_SIZE = 8
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, BATCH_SIZE)

## Model Training

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 1,
                                                      output_attentions = False,
                                                      output_hidden_states = False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = torch.optim.AdamW(model1.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss() # binary classification

In [ ]:
model.config.hidden_dropout_prob

0.1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

In [ ]:
len(train_loader.dataset)

5218

In [ ]:
for batch in train_loader:
  print(batch)
  break

{'input_ids': tensor([[ 101, 1037, 3696,  ...,    0,    0,    0],
        [ 101, 1037, 2158,  ...,    0,    0,    0],
        [ 101, 1037, 2158,  ...,    0,    0,    0],
        ...,
        [ 101, 1037, 3185,  ...,    0,    0,    0],
        [ 101, 1037, 2450,  ...,    0,    0,    0],
        [ 101, 1037, 2158,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1., 1., 0., 0., 1., 0., 1., 1.])}


In [ ]:
from tqdm import tqdm
total_correct = 0
no_batch = 0
with torch.no_grad():
  for batch in tqdm(train_loader):
      inputs = batch['input_ids']
      attention_mask = batch['attention_mask']
      labels = batch['labels']
      inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
      optimizer.zero_grad(set_to_none=True)
      outputs_dict = model1(inputs, attention_mask=attention_mask, labels=labels)
      # print(outputs_dict)
      # print(labels)
      cs = correct_answer(outputs_dict, labels)
      total_correct += cs
      no_batch += 1

print("Avg Correct per batch: ", total_correct/no_batch)
print("Accuracy: ", total_correct/(8*no_batch))

100%|██████████| 653/653 [01:29<00:00,  7.26it/s]

Avg Correct per batch:  7.6355283307810105
Accuracy:  0.9544410413476263


In [ ]:
outputs_dict.logits

tensor([[0.7283],
        [0.2760],
        [1.0249],
        [0.7577],
        [0.0969],
        [0.4580],
        [0.5662],
        [0.7712]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
def correct_answer(output_dict, labels):
  ans = [pred.detach().cpu().numpy()[0] for pred in outputs_dict.logits]
  ans = [float(0) if a < 0.5 else float(1) for a in ans]
  ans
  return np.sum(ans == labels.cpu().numpy())

In [ ]:
model.to(device)
epochs =
torch.manual_seed(12345)
for epoch in range(epochs):
    # train
    model.train()
    total_loss = 0.0
    correct = 0
    for batch in tqdm(train_loader):
        inputs = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)
        outputs_dict = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs_dict[0]
        predicted = correct_answer(outputs_dict, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Increment the number of correct predictions
        correct += predicted
    train_acc = correct / len(train_loader.dataset)
    avg_train_loss = total_loss / BATCH_SIZE

    # validate
    val_correct = 0
    model.eval()
    print("Validation:-")
    with torch.no_grad():
        for batch in tqdm(val_loader):
            inputs = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
            outputs_dict = model(inputs, attention_mask=attention_mask, labels=labels)
            val_predicted = correct_answer(outputs_dict, labels)
            val_correct += val_predicted
    val_acc = val_correct / len(val_loader.dataset)

    print(f'epoch {epoch + 1}: Avg Loss {avg_train_loss:.4f}, Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}')

100%|██████████| 653/653 [03:51<00:00,  2.82it/s]


Validation:-


100%|██████████| 73/73 [00:09<00:00,  7.87it/s]


epoch 1: Avg Loss 19.5782, Train Acc 0.5987, Val Acc 0.6431


100%|██████████| 653/653 [03:57<00:00,  2.74it/s]


...

epoch 2: Avg Loss 15.0144, Train Acc 0.7227, Val Acc 0.6259


100%|██████████| 653/653 [03:57<00:00,  2.75it/s]


Validation:-


100%|██████████| 73/73 [00:09<00:00,  7.78it/s]

epoch 3: Avg Loss 9.5598, Train Acc 0.8446, Val Acc 0.6586


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Meme Data/meme_model.pth")

In [ ]:
model1 = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 1,
                                                      output_attentions = False,
                                                      output_hidden_states = False)
model1.load_state_dict(torch.load("/content/drive/MyDrive/Meme Data/meme_model.pth"))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

## Feature Map Creation

In [ ]:
model1.classifier = nn.Identity()
model1

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model1.to(device)

In [ ]:
all_features = []
all_labels = []
with torch.no_grad():
  for batch in tqdm(train_loader):
        inputs = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
        outputs_dict = model1(inputs, attention_mask=attention_mask)

        feature = outputs_dict.logits
        feature = feature.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy().tolist()
        all_features.extend(feature)
        all_labels.extend(labels)
  features = pd.DataFrame(all_features)
  features["Labels"] = all_labels

100%|██████████| 653/653 [01:24<00:00,  7.68it/s]


In [ ]:
features

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Labels
0,-0.074023,-0.112030,-0.399132,-0.073665,0.476511,0.064854,-0.403069,0.053296,-0.019522,-0.985706,...,-0.423803,0.151787,0.220039,-0.824494,0.382869,0.225207,-0.079887,-0.167158,-0.163683,1.0
1,-0.258595,-0.172619,-0.450004,-0.057276,0.449533,0.098879,-0.417169,0.035035,-0.119147,-0.995922,...,-0.542888,0.493016,0.244294,-0.905777,0.332555,0.341171,-0.250401,-0.318815,-0.180514,1.0
2,-0.507778,-0.280582,-0.868544,0.464717,0.201069,-0.036224,0.585509,0.154884,-0.786770,-0.999435,...,-0.696650,0.843332,0.491750,-0.194759,-0.260551,0.389322,-0.321516,-0.636744,0.187596,0.0
3,0.085974,-0.011425,-0.373513,-0.237013,0.378660,0.069003,-0.599384,0.063375,0.221117,-0.959579,...,-0.271017,-0.053507,-0.042000,-0.785885,0.280219,0.158491,-0.170124,-0.094134,-0.341412,1.0
4,-0.335272,-0.121074,-0.129973,0.181269,-0.272406,-0.059930,0.335254,0.100961,-0.213555,-0.993727,...,0.020692,-0.032603,0.324002,0.167571,-0.071234,0.248293,0.237985,-0.445129,0.144760,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5213,-0.286564,-0.107031,-0.221214,0.154706,-0.199290,-0.046987,0.237557,0.000257,-0.133087,-0.993592,...,0.048544,-0.044621,0.246691,0.412899,-0.047503,0.082129,0.280149,-0.455015,0.152132,0.0
5214,-0.068129,-0.168537,-0.443262,0.022691,0.414228,0.086509,-0.460871,-0.021613,-0.165791,-0.992869,...,-0.664539,0.416661,0.160671,-0.859457,0.288509,0.267645,-0.553383,-0.135586,-0.268101,1.0
5215,-0.157512,-0.068348,-0.148111,0.173183,-0.250869,0.028327,0.310199,-0.023454,-0.010965,-0.986401,...,0.032317,-0.136008,0.221336,0.196113,0.027767,0.158546,0.144446,-0.430173,0.115392,0.0
5216,-0.171352,-0.141590,-0.420044,0.212851,-0.067831,-0.022873,0.318482,0.052490,-0.286305,-0.993837,...,-0.295179,0.201131,0.226115,0.013459,-0.128754,0.230565,-0.026624,-0.414209,-0.015029,0.0


In [ ]:
features.to_csv("/content/drive/MyDrive/Meme Data/train_features.csv")

## Prediction

In [ ]:
test_images = test_df['Img_Name'].values.tolist()

In [ ]:
ex = data.loc[(data['ImageID'] == test_images[0])]
ex

,ImageID,Labels,Caption,0
4,4.jpg,1,not_shubham14 mentally_dank kids marine drive ...,a screenshot of a man in yellow shirt and sung...


In [ ]:
ex['Caption'].values[0]

'not_shubham14 mentally_dank kids marine drive 21 years single'

In [ ]:
def Predict(img_id, tokenizer):
  ex_data = data.loc[(data['ImageID'] == img_id)]
  img_caption = ex_data['0'].values[0]
  text_caption = ex_data["Caption"].values[0]
  text = img_caption +" " + text_caption

  encoded_caption = tokenizer.encode_plus(
          text,
          add_special_tokens = True,
          max_length = 256,
          return_token_type_ids = False,
          return_attention_mask = True,
          return_tensors = 'pt',
          padding = "max_length",
          truncation = True
      )

  inputs = encoded_caption['input_ids']
  attention_mask = encoded_caption['attention_mask']
  label = ex_data['Labels'].values[0]
  label_tensor = torch.tensor([label], dtype=torch.float)
  # return inputs, attention_mask, label_tensor
  inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), label_tensor.to(device)
  with torch.no_grad():
    outputs_dict = model1(inputs, attention_mask=attention_mask)
    pred = outputs_dict.logits.detach().cpu().numpy().tolist()[0][0]
  return int(pred>0.5), label


In [ ]:
result = Predict(test_images[0], tokenizer)

In [ ]:
result

(1, 1)

In [ ]:
total = 0
correct = 0
for img_id in test_images:
  if img_id in valid_imgs:
    pred, label = Predict(img_id, tokenizer)
    if pred == label:
      correct += 1
    total += 1

In [ ]:
accuracy = correct / total
accuracy

0.9067164179104478